# Testing Naive Bayes Classifier

In this notebook, we use a method of supervised learning to attempt and classify patents into predefined categories

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

#1. load data in 

#2. preprocess data
#   - remove special characters
#   - remove stop words
#   - convert to lowercase

#3. TfidfVectorizer: Calculates Term Frequency-Inverse Document Frequency, which gives more weight to words that are important in a specific document but less common across the entire corpus

#4. split data into training and testing sets

#5. train model

#6. evaluate model

#7. make predictions



In [ ]:
# 8. visualize results

